In [1]:
# 导入包
from docplex.mp.model import Model
import pandas as pd

# 建立模型
m = Model(name='experimentmodel1.3.2')

In [2]:
# 定义参数
# 超市数量
n_shop = 8

# 仓库数量
n_warehouse = 4

# 超市索引
set_shop = range(1, n_shop + 1)

# 仓库索引
set_warehouse = range(1, n_warehouse + 1)

# 各超市到仓库的距离
distance = pd.read_csv("distance.csv")
distance.index = [i for i in set_shop]
distance.columns = [j for j in set_warehouse]

# 各超市的需求总量
demand = pd.Series(pd.read_csv("demand.csv")['demand'].values, index = [i for i in set_shop])

In [3]:
# 定义变量
x_vars = pd.DataFrame({j : pd.Series(m.binary_var(name="x_{0}_{1}".format(i,j)) 
                                     for i in set_shop) 
                       for j in set_warehouse})
x_vars.index = [i for i in set_shop]

y_vars = pd.Series([m.binary_var(name="y_{0}".format(j)) for j in set_warehouse])
y_vars.index = [j for j in set_warehouse]

maxCost = m.continuous_var(name="maxCost")

In [4]:
# 定义约束条件
unique_supply_constrains = {i : m.add_constraint(
    sum(x_vars.loc[i,j] for j in set_warehouse) == 1,
    ctname="unique_supply_constrain_{0}".format(i))
                     for i in set_shop}

selection_constraint = m.add_constraint(
    sum(y_vars.loc[j] for j in set_warehouse) == 2,
    ctname="selection_constraint")

supply_constrains = {(i,j) : m.add_constraint(
    x_vars.loc[i,j] <= y_vars.loc[j],
    ctname="supply_constrain_{0}_{1}".format(i,j))
                     for i in set_shop
                     for j in set_warehouse}

max_cost_constrains = {i : m.add_constraint(
    sum(demand.loc[i] * distance.loc[i,j] * x_vars.loc[i,j] for j in set_warehouse) <= maxCost,
    ctname="max_cost_constrain_{0}".format(i))
                      for i in set_shop}

In [5]:
# 定义目标函数
objective = maxCost

# for minimization             
m.minimize(objective)

In [6]:
# 打印模型信息
m.print_information()

Model: experimentmodel1.3.2
 - number of variables: 37
   - binary=36, integer=0, continuous=1
 - number of constraints: 49
   - linear=49
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [7]:
# 模型求解及结果输出
s = m.solve(log_output=True)
m.print_solution()

Version identifier: 20.1.0.1 | 2022-01-05 | 9df5e5246
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 49 rows, 37 columns, and 140 nonzeros.
Reduced MIP has 36 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.08 ticks)
Found incumbent of value 2200.000000 after 0.01 sec. (0.12 ticks)
Probing time = 0.00 sec. (0.03 ticks)
Cover probing fixed 0 vars, tightened 6 bounds.
Tried aggregator 3 times.
MIP Presolve eliminated 3 rows and 3 columns.
MIP Presolve modified 19 coefficients.
Aggregator did 2 substitutions.
Reduced MIP has 44 rows, 32 columns, and 109 nonzeros.
Reduced MIP has 31 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.16 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 44 rows, 32 columns, and 109 nonzeros.
Reduced MIP has 31 binaries, 1 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.09 ticks)
Probi

In [8]:
# 输出结果
for i in set_shop:
    x_vars.loc[i] = x_vars.loc[i].apply(lambda item: item.solution_value) 
y_vars = y_vars.apply(lambda item: item.solution_value) 

x_vars.to_csv("solution_x.csv")
y_vars.to_csv("solution_y.csv")

In [9]:
x_vars

,1,2,3,4
1,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,0.0
4,0.0,1.0,0.0,0.0
5,0.0,0.0,0.0,1.0
6,0.0,1.0,0.0,0.0
7,0.0,0.0,0.0,1.0
8,0.0,1.0,0.0,0.0


In [10]:
y_vars

1    0.0
2    1.0
3    0.0
4    1.0
dtype: float64